In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\felip\OneDrive\Área de Trabalho\Pós Graduação - Ciência de Dados\Projeto Aplicado\Base de Dados\base_dados.csv", sep=';', encoding='latin-1')

In [2]:
df = df.rename(columns={"Job Id": "id_vaga", "Experience":"tempo_experiencia", "Qualifications": "formacao", "Salary Range (Mín)": "salario_min",
                       "Salary Range (Max)": "salario_max", "location": "cidade", "Country":"pais", "Work Type": "tipo_trabalho", "Company Size": "tamanho_empresa",
                       "Job Posting Date":"data_vaga", "Preference":"sexo", "Contact Person":"nome_recrutador","Contact":"contato_recrutador",
                       "Job Title":"titulo_vaga", "Role":"funcao", "Job Portal":"site_vaga","Job Description":"descricao_vaga", "skills":"requisitos",
                       "Responsibilities": "responsabilidades", "Company":"empresa", "Benefits":"beneficios", "Company Profile":"perfil_empresa"})

In [3]:
df['beneficios'] = df['beneficios'].str.replace("{", "", regex=False).str.replace("}", "", regex=False)
df['beneficios']

0        'Tuition Reimbursement, Stock Options or Equit...
1        'Childcare Assistance, Paid Time Off (PTO), Re...
2        'Employee Assistance Programs (EAP), Tuition R...
3        'Employee Referral Programs, Financial Counsel...
4        'Childcare Assistance, Paid Time Off (PTO), Re...
                               ...                        
73000    'Childcare Assistance, Paid Time Off (PTO), Re...
73001    'Employee Referral Programs, Financial Counsel...
73002    'Employee Referral Programs, Financial Counsel...
73003    'Life and Disability Insurance, Stock Options ...
73004    'Transportation Benefits, Professional Develop...
Name: beneficios, Length: 73005, dtype: object

In [4]:
df = df.drop('perfil_empresa', axis = 1)

In [5]:
df['salario_min'] = df['salario_min'].str.replace('$', '', regex=False)
df['salario_max'] = df['salario_max'].str.replace('$', '', regex=False)

df['salario_min'] = df['salario_min'].str.replace('.', '', regex=False)
df['salario_max'] = df['salario_max'].str.replace('.', '', regex=False)

df['salario_min'] = df['salario_min'].str.replace(',00', '', regex=False)
df['salario_max'] = df['salario_max'].str.replace(',00', '', regex=False)

In [6]:
df['salario_min'] = df['salario_min'].astype(float)
df['salario_max'] = df['salario_max'].astype(float)

df['data_vaga'] = pd.to_datetime(df['data_vaga'], format='%d/%m/%Y')

In [7]:
df.isna().sum()

id_vaga               0
tempo_experiencia     0
formacao              0
salario_min           0
salario_max           0
cidade                0
pais                  0
tipo_trabalho         0
tamanho_empresa       0
data_vaga             0
sexo                  0
nome_recrutador       0
contato_recrutador    0
titulo_vaga           0
funcao                0
site_vaga             0
descricao_vaga        0
beneficios            0
requisitos            0
responsabilidades     0
empresa               0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73005 entries, 0 to 73004
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_vaga             73005 non-null  int64         
 1   tempo_experiencia   73005 non-null  object        
 2   formacao            73005 non-null  object        
 3   salario_min         73005 non-null  float64       
 4   salario_max         73005 non-null  float64       
 5   cidade              73005 non-null  object        
 6   pais                73005 non-null  object        
 7   tipo_trabalho       73005 non-null  object        
 8   tamanho_empresa     73005 non-null  int64         
 9   data_vaga           73005 non-null  datetime64[ns]
 10  sexo                73005 non-null  object        
 11  nome_recrutador     73005 non-null  object        
 12  contato_recrutador  73005 non-null  object        
 13  titulo_vaga         73005 non-null  object    

In [10]:
import warnings
warnings.filterwarnings("ignore")
import requests
from sqlalchemy import create_engine, text

In [25]:
user = 'root'
password = 'Fft270192!'
host = 'localhost'
string_conexao = f'mysql://{user}:{password}@{host}'

engine = create_engine(string_conexao)
conn = engine.connect()

In [27]:
with open(r"C:\Users\felip\OneDrive\Área de Trabalho\Pós Graduação - Ciência de Dados\Projeto Aplicado\MySQL\Script_MySQL.sql", 'r') as file:
    query = file.read()
conn.execute(text(query))

In [35]:
df.to_sql('stg_vagas', con=conn, schema='vagas_emprego', if_exists='replace', index=False)

73005

In [47]:
regiaoVaga = df[['cidade', 'pais']]

duplicatas = regiaoVaga.duplicated()

# Exibir as linhas duplicadas
duplicatas_linhas = regiaoVaga[duplicatas]
print(duplicatas_linhas)

               cidade        pais
21            Nairobi       Kenya
24           Chisinau     Moldova
27             London          UK
46     Port-au-Prince       Haiti
60         Georgetown      Guyana
...               ...         ...
73000       Stockholm      Sweden
73001            Apia       Samoa
73002         Mbabane    Eswatini
73003    Antananarivo  Madagascar
73004            Lima        Peru

[72789 rows x 2 columns]


In [49]:
num_duplicatas = regiaoVaga.duplicated().sum()
print(f"Número de linhas duplicadas: {num_duplicatas}")

Número de linhas duplicadas: 72789


In [55]:
regiaoVaga = regiaoVaga.drop_duplicates()
regiaoVaga['id_regiaoVaga'] = regiaoVaga.index
regiaoVaga

,cidade,pais,id_regiaoVaga
0,Yaounde,Cameroon,0
1,London,UK,1
2,Papeete,French Polynesia,2
3,George Town,Cayman Islands,3
4,Quito,Ecuador,4
...,...,...,...
1043,Bandar Seri Begawan,Brunei,1043
1080,Pristina,Kosovo,1080
1132,Praia,Cabo Verde,1132
1145,Paris,France,1145


In [57]:
infosVaga = df[['tipo_trabalho', 'titulo_vaga', 'funcao', 'descricao_vaga', 'site_vaga', 'requisitos', 'responsabilidades']]

duplicatas = infosVaga.duplicated()

# Exibir as linhas duplicadas
duplicatas_linhas = infosVaga[duplicatas]
print(duplicatas_linhas)

      tipo_trabalho               titulo_vaga                        funcao  \
54         Contract         Marketing Analyst                  Data Analyst   
76         Contract     Systems Administrator        Database Administrator   
81         Contract    Database Administrator            Database Developer   
97           Intern         Marketing Analyst                  Data Analyst   
106       Full-Time              Data Analyst                Data Scientist   
...             ...                       ...                           ...   
73000     Full-Time              Data Analyst                Data Scientist   
73001     Temporary    Database Administrator  Database Security Specialist   
73002     Part-Time         Marketing Analyst                  Data Analyst   
73003     Temporary         Marketing Analyst                  Data Analyst   
73004     Temporary  Administrative Assistant         Data Entry Specialist   

                                          descricao

In [61]:
num_duplicatas = infosVaga.duplicated().sum()
print(f"Número de linhas duplicadas: {num_duplicatas}")

Número de linhas duplicadas: 71805


In [63]:
infosVaga = infosVaga.drop_duplicates()
infosVaga['id_infosVaga'] = infosVaga.index
infosVaga

,tipo_trabalho,titulo_vaga,funcao,descricao_vaga,site_vaga,requisitos,responsabilidades,id_infosVaga
0,Temporary,Database Administrator,Database Developer,"A Database Developer designs, develops, and ma...",CareerBuilder,Database design and development SQL and databa...,"Design and implement database systems, tables,...",0
1,Intern,Data Engineer,Data Architect,A Data Architect designs and manages data infr...,Indeed,Data architecture design Database management s...,Design data architecture and systems to meet b...,1
2,Part-Time,Database Developer,SQL Database Developer,"SQL Database Developers design, implement, and...",Dice,SQL (Structured Query Language) Database desig...,"Design, develop, and maintain SQL databases, e...",2
3,Part-Time,Database Administrator,Database Developer,"A Database Developer designs, develops, and ma...",SimplyHired,Database design and development SQL and databa...,"Design and implement database systems, tables,...",3
4,Contract,Data Engineer,Big Data Engineer,A Big Data Engineer develops and manages big d...,Idealist,"Big data technologies (Hadoop, Spark, etc.) Da...","Work with large and complex datasets, using te...",4
...,...,...,...,...,...,...,...,...
11133,Intern,Research Analyst,Data Analyst Researcher,A Data Analyst Researcher conducts research an...,Dice,Data analysis techniques Research methodologie...,"Focus on data analysis, data mining, and stati...",11133
11963,Intern,Data Analyst,Data Quality Analyst,"Data Quality Analysts ensure the accuracy, com...",LinkedIn,Data quality assessment and improvement Data p...,Ensure data accuracy and quality by identifyin...,11963
13048,Temporary,Research Analyst,Data Analyst Researcher,A Data Analyst Researcher conducts research an...,Indeed,Data analysis techniques Research methodologie...,"Focus on data analysis, data mining, and stati...",13048
13773,Contract,Data Scientist,Data Engineer,Data Engineers design and maintain data pipeli...,Monster,"Data integration ETL (Extract, Transform, Load...","Design, build, and maintain data pipelines and...",13773


In [67]:
infosVaga.to_sql('dim_infosVaga', con=conn, schema='vagas_emprego', if_exists='replace', index=False)

1200

In [69]:
regiaoVaga.to_sql('dim_regiaovaga', con=conn, schema = 'vagas_emprego', if_exists='replace', index=False)

216